# KOSWAT Handleiding

Deze handleiding is geschreven voor gebruikers van KOSWAT.

Het doel van deze handleiding is om gebruikers een beeld te geven hoe invoerbestanden (.ini) worden gemaakt en wat ze precies inhouden.
In deze invoerbestanden wordt namelijk het hele project gedefinieerd in 9 tabbladen.

Deze tabbladen worden 1-voor-1 behandeld

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
plt.xkcd()
from ipyleaflet import Map, GeoData, basemaps, LayersControl, LegendControl, DrawControl
import geopandas as gpd
import leafmap
import re
import json
import os
from scripts.plotprofile_org import plot_profile, kies_dijksectie
from scripts.maak_selectie_kaart import maak_interactieve_kaart
import warnings
from shapely.geometry import shape
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipyfilechooser import FileChooser

warnings.filterwarnings("ignore")


In [ ]:
# --- JSON chooser ---
chooser_ini = FileChooser(filter_pattern="*.json")

# --- Confirm button ---
confirm_button = widgets.Button(
    description="Laad configuratiebestand",
    button_style="success",
    icon="file-text"
)
output_paths = widgets.Output()

# --- Button click handler ---
def on_confirm_click(b):
    with output_paths:
        clear_output()
        try:
            # Get selected file
            pad_ini = chooser_ini.selected
            if not pad_ini:
                print("Selecteer eerst een JSON-configuratiebestand.")
                return

            # Load JSON file
            with open(pad_ini, "r", encoding="utf-8") as f:
                config = json.load(f)

            # Save globally
            globals()['pad_ini'] = pad_ini
            globals()['config'] = config

            # Extract paths from config
            pad_vakken = config["Analyse"]["Dijksectie_Ligging"]
            pad_profielen = config["Analyse"]["Dijksectie_Invoer"]
            selectie_pad = config["Analyse"]["Dijksectie_Selectie"]

            globals()['pad_vakken'] = pad_vakken
            globals()['pad_profielen'] = pad_profielen
            globals()['selectie_pad'] = selectie_pad

            # Show results
            print("Configuratiebestand succesvol geladen!\n")
            print(f"pad_ini: {pad_ini}")
            print(f"pad_vakken: {pad_vakken}")
            print(f"pad_profielen: {pad_profielen}")

        except FileNotFoundError:
            print("Het geselecteerde bestand bestaat niet.")
        except json.JSONDecodeError as e:
            print(f"Ongeldig JSON-bestand: {e}")
        except KeyError as e:
            print(f"Vereiste sleutel ontbreekt in de configuratie: {e}")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect and display ---
confirm_button.on_click(on_confirm_click)

display(
    widgets.VBox([
        widgets.HTML("<b>Selecteer het JSON-configuratiebestand:</b>"),
        chooser_ini,
        confirm_button,
        output_paths
    ])
)

In [ ]:
# --- Button & Output ---
knop_ini = widgets.Button(
    description="Lees JSON-configuratie!",
    button_style="info",
    icon="file-text"
)
output_ini = widgets.Output()

# --- Button click function ---
def on_click_ini(b):
    with output_ini:
        clear_output()
        try:
            # Load JSON file
            with open(pad_ini, "r", encoding="utf-8") as f:
                config = json.load(f)

            # Store globally so other cells can access it
            globals()["config"] = config

            # Count sections
            num_sections = len(config.keys())

            print(f"Configuratiebestand succesvol ingelezen!")
            print(f"Aantal secties gevonden: {num_sections}")
            print(f"Secties: {', '.join(config.keys())}\n")

            # Print a preview of each section
            for section, params in config.items():
                print(f"[{section}]")
                for key, value in params.items():
                    print(f"  {key}: {value}")
                print()  # blank line between sections

        except FileNotFoundError:
            print("Het opgegeven pad naar het JSON-bestand bestaat niet.")
        except json.JSONDecodeError as e:
            print(f"Ongeldig JSON-formaat: {e}")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_ini.on_click(on_click_ini)

# --- Display button and output ---
display(knop_ini, output_ini)

# Analyse

In het eerste deel van het invoerbestand worden de dijksecties ingeladen en geselecteerd.  
De selectie van dijksecties wordt gedefinieerd.  
Per dijksectie wordt het profiel (doorsnede gegeven).  
De verschillende mate van dijkversterking (per dijksectie) worden gedefinieerd als scenario's.

Daarnaast wordt verder relevante informatie gegeven:

- De eenheidsprijzen  
- De omgevingsdatabases  
- De uitvoerfolder

![Voorbeeld analyse parameters](Figuren\Analyse.png) 

### Dijksecties

Dijksecties worden gegeven als een shapefile (lijnen) en bevatten het traject, subtraject en lengte van elk dijkvak als informatie in de tabel  
Deze informatie wordt hieronder getoond als tabel en als interactieve kaart

In [ ]:
from itables import show

# --- Button & Output ---
knop_vakken_tabel = widgets.Button(
    description="Toon dijkvakken-tabel",
    button_style="info",
    icon="table"
)
output_vakken = widgets.Output()

# --- Button click function ---
def on_click_vakken(b):
    with output_vakken:
        clear_output()
        try:
            dijksecties = gpd.read_file(pad_vakken)
            selectie_vakken = np.loadtxt(selectie_pad)
            selectie_vakken = np.array(selectie_vakken, dtype=int)
            selectie_vakken = np.array(selectie_vakken, dtype=str)
            globals()["dijksecties"] = dijksecties  # store globally
            globals()["selectie_vakken"] = selectie_vakken  # store globally

            print(f"{len(dijksecties)} dijkvakken geladen uit {pad_vakken}")
            print("Hieronder staat de interactieve tabel:")

            # Display the interactive itables table
            show(dijksecties)

        except FileNotFoundError:
            print("Het pad naar het vakkenbestand is ongeldig of het bestand bestaat niet.")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_vakken_tabel.on_click(on_click_vakken)

# --- Display button and output ---
display(knop_vakken_tabel, output_vakken)

In [ ]:
# --- Button & Output ---
knop_kaart_dijksecties = widgets.Button(
    description="Toon kaart van dijksecties",
    button_style="success",
    icon="map"
)
output_kaart = widgets.Output()

# --- Button click function ---
def on_click_kaart(b):
    with output_kaart:
        clear_output()
        try:
            if "dijksecties" not in globals():
                print("Laad eerst de dijkvakken-tabel via de vorige knop.")
                return

            # Zorg dat selectie_vakken bestaat
            if "selectie_vakken" not in globals():
                print("Laad eerst de selectie_vakken.")
                return


            # Maak de kaart
            m = leafmap.Map(center=[52.6, 6], zoom=7)
            m.layout.height = "700px"

            # Split de dijksecties op basis van selectie
            geselecteerde = dijksecties[dijksecties["Dijksectie"].isin(selectie_vakken)]
            overige = dijksecties[~dijksecties["Dijksectie"].isin(selectie_vakken)]

            # Voeg lagen toe
            m.add_gdf(
                overige,
                layer_name="Overige dijksecties",
                style={
                    "color": "red",
                    "weight": 3,
                    "fillOpacity": 0.0,
                },
            )

            m.add_gdf(
                geselecteerde,
                layer_name="Geselecteerde dijksecties",
                style={
                    "color": "yellow",
                    "weight": 4,
                    "fillOpacity": 0.1,
                },
            )

            display(m)
            print(f"{len(dijksecties)} dijksecties weergegeven op de kaart.")
            print(f"{len(geselecteerde)} geselecteerde dijksecties gemarkeerd in geel.")

        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")
# --- Connect button --- 
knop_kaart_dijksecties.on_click(on_click_kaart) 
# # --- Display button and output --- 
display(knop_kaart_dijksecties, output_kaart)

Het bestand 'dijksecties_selectie.txt' bevat de selectie van dijksecties die worden beschouwd in het KOSWAT project.    
Dit is een tekstbestand met een eigen regel voor een geselecteerde dijksectie.  

In de volgende interactieve kaart kun je een zelf een selectie maken van dijksecties voor het KOSWAT project


In [ ]:
# --- Button & Output ---
knop_selectie_kaart = widgets.Button(
    description="Selecteer dijksecties op de kaart",
    button_style="success",
    icon="map-marker"
)
output_selectie_kaart = widgets.Output()

# --- Button click function ---
def on_click_selectie_kaart(b):
    with output_selectie_kaart:
        clear_output()
        try:
            global selectie, m   # Make map + selection available to later buttons

            print("Interactieve kaart geopend — gebruik de knop links om dijksecties te selecteren.")
            print("Als je klaar bent, klik dan op 'Laat mijn selectie zien'.")

            # Create map and selection container
            m, selectie = maak_interactieve_kaart(pad_vakken)
            m.layout.height = "700px"
            display(m)

        except NameError:
            print("Zorg dat je eerst het pad naar de dijkvakken hebt ingevuld!")
        except Exception as e:
            print(f"Er is een fout opgetreden: {e}")

# --- Connect button ---
knop_selectie_kaart.on_click(on_click_selectie_kaart)

# --- Display button + output area ---
display(knop_selectie_kaart, output_selectie_kaart)

In [ ]:
# Voeg een knop toe om de selectie te laten zien
knop_selectie = widgets.Button(description="Laat mijn selectie zien", button_style='success')
output_selectie  = widgets.Output()

def on_button_click_selectie(b):
    with output_selectie:
        clear_output()
        if selectie["gdf"].empty:
            print("Maak eerst een selectie van dijksecties")
        else:
            display(selectie["gdf"])
            # Extract the Dijksectie column as strings (just in case)
            nieuwe_selectie = selectie['gdf']['Dijksectie'].astype(str).tolist()

            # Write to file, one value per line (overwrite existing)
            with open(selectie_pad, "w") as f:
                for val in nieuwe_selectie:
                    f.write(f"{val}\n")

            print(f"Bestand '{selectie_pad}' is bijgewerkt met {len(nieuwe_selectie)} dijksecties.")

knop_selectie.on_click(on_button_click_selectie)
display(knop_selectie, output_selectie)

### Dijksectie_invoer

Per dijksectie wordt de dijkdoorsnede gegeven door een .txt bestand in de folder dijksectie_invoer. 
Zo'n bestand ziet er als volgt uit:

![Voorbeeld analyse parameters](Figuren\Dijkprofiel.png)    

Hieronder wordt, stap voor stap een figuur geschetst van dit dijkprofiel

In [ ]:
button_profiel = widgets.Button(
    description="Toon profiel",
    button_style="success",  # 'success', 'info', 'warning', 'danger'
    icon='play'              # optional icon
)
output_profiel = widgets.Output()

def on_button_click_profiel(b):
    with output_profiel:
        clear_output()
        try:
            selectie_dijksectie = selectie['gdf']
            kies_dijksectie(selectie_dijksectie, plot_profile, pad_profielen)
        except NameError:
            print("Maak eerst een selectie van dijksecties")

button_profiel.on_click(on_button_click_profiel)

display(button_profiel, output_profiel)

# Versterking scenario's
De volgende invoerparameters zijn de versterkingsscenario's. Deze worden, net zoals het dijkprofiel gegeven in een .ini bestand per dijksectie. Dat ziet er als volgt uit   

![Voorbeeld scenarios](Figuren\Scenarios.png)           

Per scenario zijn er drie versterkingsparameters:   
- dH: De hoogte opgave
- dS: De stabiliteits opgave 
- dP: De piping opgave

Met deze opgaven rekent KOSWAT de maatregel in grond door   
Dit is weergegeven in het volgende figuur:   
![Doorsnede scenario](Figuren\Scenario_Toelichting.png)



# Omgevingsdatabases

Deze grondmaatregel is in principe de goedkoopste oplossing en heeft daarvoor altijd de voorkeur. Alleen is er niet op elke plek ruimte voor deze maatregel     
Om te bepalen waar er wel of geen ruimte is voor de grondmaatregel werkt KOSWAT met omgevingsdatabases, deze bevatten informatie over de beschikbare ruimte langs de dijk   

De omgevingsdatabases zijn gegeven in de categorieën bebouwing, infrastructuur en water   
Per categorie is er een .csv bestand beschikbaar met daarin de afstand tot het dichtstbijzijndste object van die categorie, zowel binnen-als-buitendijks

In [ ]:
omgevingsdatabase_voorbeeld = pd.read_csv(r"c:\Users\stouten\KOSWAT\Databases\Omgeving_bebouwing.csv", delimiter=';')

display(omgevingsdatabase_voorbeeld)